In [44]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sa2022/Sample__submission.csv
/kaggle/input/sa2022/train.csv
/kaggle/input/sa2022/test.csv


In [45]:
import numpy as np
import pandas as pd

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [46]:
train_df = pd.read_csv("/kaggle/input/sa2022/train.csv")
test_df = pd.read_csv("/kaggle/input/sa2022/test.csv")

In [47]:
print(test_df.shape)
print(train_df.shape)

(5244, 14)
(15730, 16)


In [48]:
test_df.isnull().sum()

id              0
title           0
Rating        203
maincateg      67
platform        0
actprice1       0
norating1       0
noreviews1      0
star_5f        68
star_4f         0
star_3f         0
star_2f         0
star_1f       186
fulfilled1      0
dtype: int64

In [49]:
train_df.head(10)

,id,title,Rating,maincateg,platform,price1,actprice1,Offer %,norating1,noreviews1,star_5f,star_4f,star_3f,star_2f,star_1f,fulfilled1
0,16695,Fashionable & Comfortable Bellies For Women (...,3.9,Women,Flipkart,698,999,30.13%,38.0,7.0,17.0,9.0,6.0,3,3,0
1,5120,Combo Pack of 4 Casual Shoes Sneakers For Men ...,3.8,Men,Flipkart,999,1999,50.03%,531.0,69.0,264.0,92.0,73.0,29,73,1
2,18391,Cilia Mode Leo Sneakers For Women (White),4.4,Women,Flipkart,2749,4999,45.01%,17.0,4.0,11.0,3.0,2.0,1,0,1
3,495,Men Black Sports Sandal,4.2,Men,Flipkart,518,724,15.85%,46413.0,6229.0,1045.0,12416.0,5352.0,701,4595,1
4,16408,Men Green Sports Sandal,3.9,Men,Flipkart,1379,2299,40.02%,77.0,3.0,35.0,21.0,7.0,7,7,1
5,2692,Women Pink Flats Sandal,4.1,Women,Flipkart,359,499,28.06%,274.0,25.0,153.0,49.0,34.0,15,23,0
6,3236,Women Navy Heels Sandal,4.0,Women,Flipkart,499,999,50.05%,249.0,31.0,130.0,52.0,27.0,9,28,1
7,14633,Women Blue Wedges Sandal,3.9,Women,Flipkart,209,1299,83.91%,121.0,17.0,63.0,25.0,14.0,9,12,0
8,1872,Sneakers For Women (Pink),4.2,Women,Flipkart,549,999,45.05%,191.0,30.0,120.0,25.0,31.0,7,12,1
9,1668,"Men Black, Beige Sandal",4.1,Men,Flipkart,809,899,10.01%,36126.0,5028.0,18688.0,9899.0,3709.0,1272,2558,0


In [50]:
train_df.columns

Index(['id', 'title', 'Rating', 'maincateg', 'platform', 'price1', 'actprice1',
       'Offer %', 'norating1', 'noreviews1', 'star_5f', 'star_4f', 'star_3f',
       'star_2f', 'star_1f', 'fulfilled1'],
      dtype='object')

In [51]:
y=train_df.price1
id=test_df.id
train=train_df.drop(['price1','id','Offer %','title'],axis='columns')
test=test_df.drop(['id','title'],axis='columns')
data=pd.concat([train,test],axis=0)

In [52]:
data.head(20)

,Rating,maincateg,platform,actprice1,norating1,noreviews1,star_5f,star_4f,star_3f,star_2f,star_1f,fulfilled1
0,3.9,Women,Flipkart,999,38.0,7.0,17.0,9.0,6.0,3,3.0,0
1,3.8,Men,Flipkart,1999,531.0,69.0,264.0,92.0,73.0,29,73.0,1
2,4.4,Women,Flipkart,4999,17.0,4.0,11.0,3.0,2.0,1,0.0,1
3,4.2,Men,Flipkart,724,46413.0,6229.0,1045.0,12416.0,5352.0,701,4595.0,1
4,3.9,Men,Flipkart,2299,77.0,3.0,35.0,21.0,7.0,7,7.0,1
5,4.1,Women,Flipkart,499,274.0,25.0,153.0,49.0,34.0,15,23.0,0
6,4.0,Women,Flipkart,999,249.0,31.0,130.0,52.0,27.0,9,28.0,1
7,3.9,Women,Flipkart,1299,121.0,17.0,63.0,25.0,14.0,9,12.0,0
8,4.2,Women,Flipkart,999,191.0,30.0,120.0,25.0,31.0,7,12.0,1
9,4.1,Men,Flipkart,899,36126.0,5028.0,18688.0,9899.0,3709.0,1272,2558.0,0


In [53]:
nan_cols = [i for i in data.columns if data[i].isnull().any()]
nan_cols

['Rating',
 'maincateg',
 'norating1',
 'noreviews1',
 'star_5f',
 'star_4f',
 'star_3f',
 'star_1f']

In [54]:
data['maincateg'].value_counts()

Women    11813
Men       8568
Name: maincateg, dtype: int64

In [55]:
# data[num].isna().sum()

In [56]:
data=data.fillna({
       'maincateg':'Women'
})
#data['noreviews1']=data['noreviews1'].fillna(data['noreviews1'].median())

In [57]:
def add(row):
   return row[4]-(row[7]+row[8]+row[9]+row[10])

In [58]:
def add_1(row):
   return row[4]-(row[7]+row[8]+row[9]+row[6])

In [59]:
def add_3(row):
   return row[4]-(row[7]+row[10]+row[9]+row[6])

In [60]:
def add_4(row):
   return row[4]-(row[10]+row[8]+row[9]+row[6])

In [61]:
# def fit_1(row):
#     if row[4] < 300:
#                      return (row[4]-(row[8]+row[9]+row[7]))

In [62]:
def fit_21(row):
    return (row[4]-(row[8]+row[9]+row[7]))*0.2

In [63]:
def rat(row):
    s=5*row[6]+4*row[7]+3*row[8]+2*row[9]+row[10]
    return s/row[4]

In [64]:
def rat_1(row):
    s=5*row[6]+4*row[7]+3*row[8]+2*row[9]+row[10]
    return s/row[1]

In [65]:
def tot(row):
   return (row[10]+row[8]+row[9]+row[6]+row[7])

In [66]:
def rf_5(row):
    if row[0] != 5:
                   g=row[0]*(row[7]+row[8]+row[9]+row[10])-(4*row[7]+3*row[8]+2*row[9]+row[10])
                   return g/(5-row[0])
    else :
             return 0

In [67]:
def rf_4(row):
    if row[0] != 4:
                   g=row[0]*(row[6]+row[8]+row[9]+row[10])-(5*row[6]+3*row[8]+2*row[9]+row[10])
                   return g/(4-row[0])
    else :
             return 0

In [68]:
def rf_3(row):
    if row[0] != 3:
                   g=row[0]*(row[6]+row[7]+row[9]+row[10])-(5*row[6]+4*row[7]+2*row[9]+row[10])
                   return g/(3-row[0])
    else :
             return 0

In [69]:
def rft_5(row):
    if (row[8]!=np.NaN):
                      if row[0] != 4.5:
                            g=row[0]*(row[8]+row[9]+row[10])-(3*row[8]+2*row[9]+row[10])
                            return g/(9-2*row[0])
                      else:
                            return 0

In [70]:
def rft_4(row):
    if (row[6]!=np.NaN):
                      if row[0] != 3.5:
                            g=row[0]*(row[6]+row[9]+row[10])-(5*row[6]+2*row[9]+row[10])
                            return g/(7-2*row[0])
                      else:
                            return 0

In [71]:
def rft_3(row):
    if (row[7]!=np.NaN):
                      if row[0] != 4:
                            g=row[0]*(row[7]+row[9]+row[10])-(4*row[7]+2*row[9]+row[10])
                            return g/(8-2*row[0])
                      else:
                            return 0

In [72]:
def rftt_3(row):
                 if row[0] != 4:
                            g=row[0]*(row[9]+row[10])-(2*row[9]+row[10])
                            return g/(12-3*row[0])
                 else:
                            return 0
                      

In [73]:
data.star_5f=data.star_5f.fillna(data.apply(add, axis=1))
data.star_1f=data.star_1f.fillna(data.apply(add_1, axis=1))
data.star_3f=data.star_3f.fillna(data.apply(add_3, axis=1))
data.star_4f=data.star_4f.fillna(data.apply(add_4, axis=1))

#data.star_1f=data.star_1f.fillna(data.apply(fit_1, axis=1))
data[data.star_1f.isnull()]

,Rating,maincateg,platform,actprice1,norating1,noreviews1,star_5f,star_4f,star_3f,star_2f,star_1f,fulfilled1
35,NaN,Women,Flipkart,999,473.0,68.0,NaN,119.0,45.0,16,NaN,1
90,NaN,Men,Flipkart,2499,52.0,5.0,NaN,13.0,6.0,4,NaN,1
391,2.8,Men,Flipkart,899,4.0,0.0,NaN,1.0,0.0,0,NaN,0
470,NaN,Women,Flipkart,999,1877.0,252.0,NaN,451.0,243.0,125,NaN,1
479,NaN,Women,Flipkart,1299,26.0,4.0,NaN,5.0,4.0,2,NaN,0
776,NaN,Men,Flipkart,499,627.0,63.0,NaN,135.0,75.0,40,NaN,1
866,NaN,Men,Flipkart,2999,280.0,43.0,NaN,60.0,36.0,21,NaN,1
978,NaN,Men,Flipkart,998,115.0,14.0,NaN,19.0,11.0,15,NaN,1
1019,NaN,Women,Flipkart,1999,5.0,1.0,NaN,1.0,1.0,1,NaN,0
1033,3.8,Men,Flipkart,499,29.0,3.0,NaN,7.0,5.0,1,NaN,0


In [74]:
#data.star_1f=data.star_1f.fillna(data.apply(fit_21, axis=1))
data.star_5f=data.star_5f.fillna(data.apply(add, axis=1))


data[data.star_5f.notnull() & data.norating1.isnull()& data.star_4f.notnull()&data.star_3f.notnull()]

,Rating,maincateg,platform,actprice1,norating1,noreviews1,star_5f,star_4f,star_3f,star_2f,star_1f,fulfilled1
19,4.0,Women,Flipkart,999,NaN,NaN,28.0,9.0,10.0,2,5.0,0
116,4.3,Women,Flipkart,4999,NaN,NaN,46.0,27.0,5.0,1,5.0,1
2306,4.0,Women,Flipkart,299,NaN,NaN,17.0,14.0,7.0,1,3.0,1
2336,4.2,Women,Flipkart,799,NaN,NaN,1206.0,521.0,241.0,87,89.0,0
2562,4.2,Women,Flipkart,999,NaN,1028.0,3773.0,1498.0,593.0,227,416.0,0
3027,4.0,Women,Flipkart,999,NaN,NaN,369.0,145.0,105.0,34,67.0,1
3331,4.0,Women,Flipkart,999,NaN,NaN,14.0,0.0,3.0,1,3.0,1
4577,4.2,Men,Flipkart,1049,NaN,NaN,11192.0,4505.0,2003.0,756,1316.0,1
5105,3.9,Men,Flipkart,2499,NaN,NaN,2539.0,1192.0,701.0,357,631.0,1
5239,4.2,Women,Flipkart,999,NaN,45.0,140.0,69.0,20.0,9,14.0,1


In [75]:
data.norating1=data.norating1.fillna(data.apply(tot, axis=1))
#data.Rating=data.Rating.fillna(data.apply(rat_1, axis=1))

In [76]:
data.star_5f=data.star_5f.fillna(data.apply(rf_5, axis=1))
data.star_4f=data.star_4f.fillna(data.apply(rf_4, axis=1))
data.star_3f=data.star_3f.fillna(data.apply(rf_3, axis=1))

data.norating1=data.norating1.fillna(data.apply(tot, axis=1))

In [77]:
nan_cols = [i for i in data.columns if data[i].isnull().any()]
nan_cols

['Rating',
 'norating1',
 'noreviews1',
 'star_5f',
 'star_4f',
 'star_3f',
 'star_1f']

In [78]:
data.isnull().sum()

Rating        203
maincateg       0
platform        0
actprice1       0
norating1     428
noreviews1    578
star_5f       504
star_4f       374
star_3f       211
star_2f         0
star_1f        45
fulfilled1      0
dtype: int64

In [79]:
data[ data.norating1.isnull()]

,Rating,maincateg,platform,actprice1,norating1,noreviews1,star_5f,star_4f,star_3f,star_2f,star_1f,fulfilled1
63,3.9,Men,Flipkart,1500,NaN,NaN,NaN,NaN,10.0,5,9.0,1
68,4.1,Women,Flipkart,999,NaN,27.0,NaN,NaN,22.0,5,15.0,1
91,3.6,Women,Flipkart,249,NaN,NaN,NaN,NaN,466.0,281,476.0,1
131,4.2,Women,Flipkart,3799,NaN,NaN,NaN,NaN,114.0,38,73.0,1
164,3.8,Women,Flipkart,999,NaN,2.0,NaN,NaN,5.0,1,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
15465,4.2,Men,Flipkart,2499,NaN,NaN,NaN,9.0,NaN,2,2.0,0
15538,4.0,Men,Flipkart,2400,NaN,49.0,NaN,73.0,NaN,26,35.0,1
15674,4.3,Women,Flipkart,3999,NaN,NaN,NaN,NaN,268.0,102,140.0,1
15709,3.9,Women,Flipkart,5499,NaN,NaN,NaN,NaN,6.0,2,10.0,1


In [ ]:
data.star_5f=data.star_5f.fillna(data.apply(rf_5, axis=1))
data.norating1=data.norating1.fillna(data.apply(tot, axis=1))

In [80]:
data.isnull().sum()

Rating        203
maincateg       0
platform        0
actprice1       0
norating1     428
noreviews1    578
star_5f       504
star_4f       374
star_3f       211
star_2f         0
star_1f        45
fulfilled1      0
dtype: int64

In [ ]:
# data.star_5f=data.star_5f.fillna(data.apply(rft_5, axis=1))
# data.star_4f=data.star_4f.fillna(data.apply(rf_4, axis=1))
# data.norating1=data.norating1.fillna(data.apply(tot, axis=1))

In [81]:
data.isnull().sum()

Rating        203
maincateg       0
platform        0
actprice1       0
norating1     428
noreviews1    578
star_5f       504
star_4f       374
star_3f       211
star_2f         0
star_1f        45
fulfilled1      0
dtype: int64

In [ ]:
# data.star_4f=data.star_4f.fillna(data.apply(rft_4, axis=1))
# data.star_3f=data.star_3f.fillna(data.apply(rf_3, axis=1))
# data.norating1=data.norating1.fillna(data.apply(tot, axis=1))

In [82]:
data.isnull().sum()

Rating        203
maincateg       0
platform        0
actprice1       0
norating1     428
noreviews1    578
star_5f       504
star_4f       374
star_3f       211
star_2f         0
star_1f        45
fulfilled1      0
dtype: int64

In [ ]:
# data.star_3f=data.star_3f.fillna(data.apply(rft_3, axis=1))
# data.star_5f=data.star_5f.fillna(data.apply(rf_5, axis=1))
# data.norating1=data.norating1.fillna(data.apply(tot, axis=1))

In [83]:
data.isnull().sum()

Rating        203
maincateg       0
platform        0
actprice1       0
norating1     428
noreviews1    578
star_5f       504
star_4f       374
star_3f       211
star_2f         0
star_1f        45
fulfilled1      0
dtype: int64

In [ ]:
# data.star_3f=data.star_3f.fillna(data.apply(rftt_3, axis=1))
# data.star_5f=data.star_5f.fillna(data.apply(rft_5, axis=1))
# data.star_4f=data.star_4f.fillna(data.apply(rf_4, axis=1))
# data.norating1=data.norating1.fillna(data.apply(tot, axis=1))

In [84]:
data[ data.norating1.isnull()]

,Rating,maincateg,platform,actprice1,norating1,noreviews1,star_5f,star_4f,star_3f,star_2f,star_1f,fulfilled1
63,3.9,Men,Flipkart,1500,NaN,NaN,NaN,NaN,10.0,5,9.0,1
68,4.1,Women,Flipkart,999,NaN,27.0,NaN,NaN,22.0,5,15.0,1
91,3.6,Women,Flipkart,249,NaN,NaN,NaN,NaN,466.0,281,476.0,1
131,4.2,Women,Flipkart,3799,NaN,NaN,NaN,NaN,114.0,38,73.0,1
164,3.8,Women,Flipkart,999,NaN,2.0,NaN,NaN,5.0,1,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
15465,4.2,Men,Flipkart,2499,NaN,NaN,NaN,9.0,NaN,2,2.0,0
15538,4.0,Men,Flipkart,2400,NaN,49.0,NaN,73.0,NaN,26,35.0,1
15674,4.3,Women,Flipkart,3999,NaN,NaN,NaN,NaN,268.0,102,140.0,1
15709,3.9,Women,Flipkart,5499,NaN,NaN,NaN,NaN,6.0,2,10.0,1


In [85]:
data[ data.norating1.isnull() &  data.star_3f.isnull() & data.star_4f.isnull() &data.star_5f.isnull()]

,Rating,maincateg,platform,actprice1,norating1,noreviews1,star_5f,star_4f,star_3f,star_2f,star_1f,fulfilled1
728,3.7,Women,Flipkart,998,NaN,4.0,NaN,NaN,NaN,3,4.0,1
857,4.1,Men,Flipkart,999,NaN,NaN,NaN,NaN,NaN,720,1213.0,1
1115,3.5,Women,Flipkart,995,NaN,NaN,NaN,NaN,NaN,1,0.0,1
1150,4.1,Women,Flipkart,999,NaN,151.0,NaN,NaN,NaN,42,88.0,0
1283,4.4,Women,Flipkart,1199,NaN,17.0,NaN,NaN,NaN,3,2.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
14760,4.4,Women,Flipkart,4099,NaN,NaN,NaN,NaN,NaN,2,7.0,1
14847,3.9,Women,Flipkart,599,NaN,NaN,NaN,NaN,NaN,35,66.0,0
14871,4.2,Women,Amazon,2997,NaN,NaN,NaN,NaN,NaN,13,65.0,1
15013,4.2,Women,Flipkart,999,NaN,NaN,NaN,NaN,NaN,24,43.0,0


In [86]:
no=data.norating1
rat=data.Rating
data=data.drop(['norating1'],axis='columns')

In [88]:
from sklearn.impute import KNNImputer
knn = KNNImputer(n_neighbors=3)
num = [col for col in data.columns if data[col].dtypes != 'O']
data[num].head()

,Rating,actprice1,noreviews1,star_5f,star_4f,star_3f,star_2f,star_1f,fulfilled1
0,3.9,999,7.0,17.0,9.0,6.0,3,3.0,0
1,3.8,1999,69.0,264.0,92.0,73.0,29,73.0,1
2,4.4,4999,4.0,11.0,3.0,2.0,1,0.0,1
3,4.2,724,6229.0,1045.0,12416.0,5352.0,701,4595.0,1
4,3.9,2299,3.0,35.0,21.0,7.0,7,7.0,1


In [89]:
data[num].corr()

,Rating,actprice1,noreviews1,star_5f,star_4f,star_3f,star_2f,star_1f,fulfilled1
Rating,1.000000,0.242678,0.019945,0.024440,0.028742,-0.000920,-0.025993,-0.034120,0.173868
actprice1,0.242678,1.000000,-0.063003,-0.062539,-0.056248,-0.070687,-0.080678,-0.075578,0.230858
noreviews1,0.019945,-0.063003,1.000000,0.985712,0.984357,0.984018,0.946444,0.883040,0.116008
star_5f,0.024440,-0.062539,0.985712,1.000000,0.976172,0.976792,0.955454,0.891217,0.124887
star_4f,0.028742,-0.056248,0.984357,0.976172,1.000000,0.964427,0.916022,0.837003,0.116942
star_3f,-0.000920,-0.070687,0.984018,0.976792,0.964427,1.000000,0.974112,0.917133,0.116230
star_2f,-0.025993,-0.080678,0.946444,0.955454,0.916022,0.974112,1.000000,0.956245,0.120701
star_1f,-0.034120,-0.075578,0.883040,0.891217,0.837003,0.917133,0.956245,1.000000,0.118185
fulfilled1,0.173868,0.230858,0.116008,0.124887,0.116942,0.116230,0.120701,0.118185,1.000000


In [94]:
from sklearn import linear_model
lasso_reg = linear_model.Lasso(alpha=100, max_iter=100, tol=0.1)
#ridge= Ridge(alpha=100, max_iter=100, tol=0.1)
imp=IterativeImputer(estimator=lasso_reg, missing_values=np.nan, max_iter=5)


In [96]:
#data[num]=knn.fit_transform(data[num])

In [97]:
data[num].isna().sum()

Rating        0
actprice1     0
noreviews1    0
star_5f       0
star_4f       0
star_3f       0
star_2f       0
star_1f       0
fulfilled1    0
dtype: int64

In [98]:
data=pd.concat([no,data],axis='columns')
data.norating1=data.norating1.fillna(data.apply(tot, axis=1))
#data.Rating=data.Rating.fillna(data.apply(rat, axis=1))

In [99]:
data.isnull().sum()

norating1     0
Rating        0
maincateg     0
platform      0
actprice1     0
noreviews1    0
star_5f       0
star_4f       0
star_3f       0
star_2f       0
star_1f       0
fulfilled1    0
dtype: int64

In [102]:
dataset = pd.get_dummies(data, drop_first=True)
#dataset=dataset.drop(['norating1'],axis='columns')

In [103]:
X=dataset.iloc[:15730,:]
test=dataset.iloc[15730:,:]

In [104]:
X.tail()

,norating1,Rating,actprice1,noreviews1,star_5f,star_4f,star_3f,star_2f,star_1f,fulfilled1,maincateg_Women,platform_Flipkart
15725,807.000000,4.3,1199.0,114.0,485.000000,177.000000,61.0,41.0,43.0,0.0,1,1
15726,246.000000,3.9,998.0,34.0,120.000000,45.000000,37.0,16.0,28.0,1.0,1,1
15727,204.738003,3.7,499.0,18.0,131.209125,28.528877,20.0,10.0,15.0,1.0,1,1
15728,750.000000,3.9,4499.0,479.0,13.000000,6.000000,10.0,25.0,47.0,1.0,0,0
15729,8.000000,5.0,2799.0,1.0,8.000000,0.000000,0.0,0.0,0.0,1.0,0,1


In [105]:
y.tail()

15725     567
15726     499
15727     329
15728    2249
15729    1539
Name: price1, dtype: int64

In [106]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1)

In [113]:
from sklearn.model_selection import StratifiedKFold
folds = StratifiedKFold(n_splits=5)
from sklearn.model_selection import cross_val_score

In [112]:
# from sklearn.ensemble import GradientBoostingClassifier #For Regression
# cl = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1)
# cl.fit(X_train, y_train)
# cl.score(X_test,y_test)

In [114]:
from sklearn import linear_model
lin=linear_model.LinearRegression()
lasso_reg = linear_model.Lasso(alpha=100, max_iter=100, tol=0.1)
lasso_reg.fit(X_train, y_train)
lasso_reg.score(X_test,y_test)

0.7736009379731564

In [115]:
cross_val_score(lasso_reg,X,y)

array([0.81222291, 0.78878914, 0.79799122, 0.77542342, 0.78535122])

In [116]:
from sklearn.linear_model import Ridge
ridge_reg= Ridge(alpha=10, max_iter=100, tol=0.2)
ridge_reg.fit(X_train, y_train)
ridge_reg.score(X_test,y_test)

0.7871945889178327

In [117]:
cross_val_score(ridge_reg,X,y)

array([0.82081169, 0.80104092, 0.80596458, 0.78538898, 0.795218  ])

In [118]:
lin.fit(X_train, y_train)
lin.score(X_test,y_test)

0.7871823768350077

In [119]:
cross_val_score(lin,X,y)

array([0.82079199, 0.8011183 , 0.80597433, 0.78539441, 0.79516304])

In [120]:
# from catboost import CatBoostClassifier
# from sklearn.metrics import accuracy_score

In [121]:
# categorical_var = np.where(X_train.dtypes != np.float)[0]
# model = CatBoostClassifier(iterations=50)
# model.fit(X_train, y_train,cat_features = categorical_var,plot=False)

In [ ]:
# model.score(X_test,y_test)

Submission file

In [122]:
predicted=lasso_reg.predict(test)

In [123]:
prediction = pd.DataFrame(predicted,columns=['price1']).to_csv('submissio.csv',index=False)
result=pd.read_csv('submissio.csv')
result=pd.concat([id,result],axis='columns')
result.tail()

,id,price1
5239,14033,376.143307
5240,297,980.235602
5241,18733,512.532386
5242,6162,270.895829
5243,13574,400.538049


In [124]:
prediction = pd.DataFrame(result, columns=['id','price1']).to_csv('submission.csv',index=False)
results=pd.read_csv('submission.csv')
results.head(20)

,id,price1
0,2242,457.140478
1,20532,278.902174
2,10648,514.332618
3,20677,1448.664351
4,12593,514.232328
5,11159,516.877829
6,10680,1168.362167
7,6433,2057.004988
8,2794,279.535061
9,11579,1115.476243
